# index进阶操作
下面介绍的方法只支持部分Index类型。

## 从index中恢复出原始数据
给定id，可以使用reconstruct或者reconstruct_n方法从index中回复出原始向量。  
支持IndexFlat, IndexIVFFlat (需要与make_direct_map结合), IndexIVFPQ, IndexPreTransform这几类索引类型。

In [6]:
# 导入faiss
import sys
import numpy as np 
import faiss

#生成数据
d = 16
n_data = 500
data = np.random.rand(n_data, d).astype('float32')

In [5]:
index = faiss.IndexFlatL2(d)
index.add(data)
re_data = index.reconstruct(0)  #指定需要恢复的向量的id,每次只能恢复一个向量
print(re_data)
re_data_n = index.reconstruct_n(0, 10) # 从第0个向量开始，连续取10个
print(re_data_n.shape)

[0.83482474 0.6438837  0.24734788 0.6537704  0.61774486 0.9384649
 0.12293569 0.80372137 0.6918489  0.6864977  0.6374077  0.52337897
 0.09799734 0.14641039 0.5328222  0.88016546]
(10, 16)


## 从index中移除向量
使用remove_ids方法可以移除index中的部分向量，调用了IDSelector对象（或IDSelectorBatch批量操作）标识每个向量是否应该被移除。因为要遍历标识数据库中的每一个向量，所以只有在需要移除大部分向量时才建议使用。   
支持IndexFlat, IndexIVFFlat, IndexIVFPQ, IDMap。

In [11]:
index = faiss.IndexFlatL2(d)
index.add(data)
print(index.ntotal)
# index.remove_ids(np.array([0]))
index.remove_ids(np.arange(5)) # 需要移除的向量的id
print(index.ntotal)  #移除了5个向量，还剩495个

500
495


## 搜索距离范围内的向量
以查询向量为中心，返回距离在一定范围内的结果，如返回数据库中与查询向量距离小于0.3的结果。  
支持IndexFlat, IndexIVFFlat，只支持在CPU使用。

In [17]:
index = faiss.IndexFlatL2(d)
index.add(data)
dist = float(np.linalg.norm(data[3] - data[0])) * 0.99  # 定义一个半径/阈值
print(dist)
_, dis, ind = index.range_search(data[[49], :], dist)  # 用第50个向量查询
print(dis)  # 返回结果是一个三元组，分别是limit(返回的结果的数量), distance, index

_, dis, ind = index.range_search(data[[9], :], dist)  # 用第10个向量查询
print(dis)  # 返回结果是一个三元组，分别是limit(返回的结果的数量), distance, index

1.9466463339328766
[1.5384469 1.4711096 1.6992308 1.3599523 1.8639128 1.8728075 1.816922
 0.        1.6398396 1.6408762 1.5378789 1.6585189 1.590842  1.8358855
 1.7690482 1.6529568 1.6374359 1.9222212 1.7712178 1.7988551 1.6408085
 1.3410411 1.6889251 1.9464017 1.7024078 1.6522535 1.8238969 1.8631847
 1.5418944 1.670974  1.6715947 1.5081348 1.9099058 1.8395643 1.4895053
 1.8079453 1.934546  0.6599543 1.6656785 1.8234447 1.8075504 1.548338
 1.6937177 1.6249175 1.729368  1.3271422 1.7416501 1.3548261 1.0927346
 1.7518137 1.7552993 1.896307  1.9200239 1.6671427 1.2523265 1.1999724
 1.8288059 1.6901855 1.3587701 1.9093046 1.6236575 1.6918842 1.9014039
 1.8057953 1.9295995 1.8126392]
[1.6781944  1.6698971  0.         1.905075   1.2732563  1.8204238
 0.7628526  1.7816186  1.301338   1.6780018  1.6545635  1.6984217
 1.4493203  1.6713542  1.537207   1.5003004  1.6203446  0.824362
 1.4711096  1.5239989  1.6029797  1.6971319  1.337048   1.7034856
 1.2767963  1.7624524  1.901829   1.3511816  1.71

## 拆分/合并index
可以将多个index合并，需要注意的是，多个Index的数据应该满足同一分布，并且用同一分布的数据训练index，如果多个Index的数据分布不同，合并时并不会报错，但在理论上会降低索引的精度，应该用与合并后的数据集同分布的训练集再次训练。

In [34]:
nlist = 10
quantizer = faiss.IndexFlatL2(d)
index1 = faiss.IndexIVFFlat(quantizer, d, nlist)
index1.train(data)
index1.add(data[:250])

index2 = faiss.IndexIVFFlat(quantizer, d, nlist)
index2.train(data)
index2.add(data[250:])

index1.merge_from(index2, 250)  # merger_from(other_index, add_id)  add_id 控制新增元素的下标id
print(index1.ntotal)  # 合并后应该包含500个向量

dis, ind = index1.search(data[:5], 10)
print(ind)

500
[[  0 454  12 345 278  95 306 296 161 322]
 [  1  53 477 455  36  63 337 140 287 138]
 [  2  99 113  63 479 229  36 337 173   1]
 [  3 492 197 478 484   5 152 272 271  85]
 [  4 346   8 194 312 215  69 342 126 260]]
